# Part I - Prosper Loan Data Exploration 
## by Arthur Ezenwanne 
## Preliminary Wrangling


This data set contains 113,937 loans with 81 variables on each loan, including loan amount, borrower rate (or interest rate), current loan status, borrower income, and many others.

In [1]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from pandas.api.types import CategoricalDtype

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

In [2]:
# load in the dataset into a pandas dataframe, print statistics
df = pd.read_csv('prosperLoanData.csv')

In [3]:
# high-level overview of data shape and composition
display(df.shape)
display(df.sample(5))

(113937, 81)

,ListingKey,ListingNumber,ListingCreationDate,CreditGrade,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,...,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors
26745,34E335311421355064BD4B1,538439,2011-11-10 12:29:46.320000000,NaN,36,Current,NaN,0.20564,0.1764,0.1664,...,-123.75,0.0,0.0,0.0,0.0,1.0,0,0,0.0,160
17249,591734200353985138D7A33,321075,2008-04-29 09:34:51.247000000,C,36,Completed,2011-05-07 00:00:00,0.12562,0.1045,0.0945,...,-32.52,0.0,0.0,0.0,0.0,1.0,0,0,0.0,36
90984,8709357422545975709B8E3,741380,2013-04-01 21:58:16.160000000,NaN,60,Current,NaN,0.21566,0.1914,0.1814,...,-76.12,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1
49624,1CFC3551987220161DA3121,613750,2012-07-18 10:05:15.840000000,NaN,60,Completed,2012-11-13 00:00:00,0.27462,0.2489,0.2389,...,-9.36,0.0,0.0,0.0,0.0,1.0,0,0,0.0,75
18275,4AC23432580801124E72602,403341,2008-09-24 06:39:17.357000000,E,36,Completed,2011-10-06 00:00:00,0.41355,0.3500,0.3400,...,-17.96,0.0,0.0,0.0,0.0,1.0,0,0,0.0,14


In [4]:
# view info about the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 81 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   ListingKey                           113937 non-null  object 
 1   ListingNumber                        113937 non-null  int64  
 2   ListingCreationDate                  113937 non-null  object 
 3   CreditGrade                          28953 non-null   object 
 4   Term                                 113937 non-null  int64  
 5   LoanStatus                           113937 non-null  object 
 6   ClosedDate                           55089 non-null   object 
 7   BorrowerAPR                          113912 non-null  float64
 8   BorrowerRate                         113937 non-null  float64
 9   LenderYield                          113937 non-null  float64
 10  EstimatedEffectiveYield              84853 non-null   float64
 11  EstimatedLoss

The dataset contains a lot of features which may make it difficult to appropriately show relationships between features. I want to focus my exploration on about 15-20 features of interest. These are features that would aid me in answering some key questions regarding the relationship between loan outcomes, borrower's interest rate, and effect of borrower's eeconomic status such as home owner, employment status, etc on the loan amount.

In [5]:
# select the desired columns subset
cols = ['Term', 'LoanStatus', 'BorrowerAPR', 'BorrowerRate', 'EstimatedReturn',  
        'ProsperRating (Alpha)', 'ListingCategory (numeric)', 'EmploymentStatus', 'EmploymentStatusDuration', 
        'IsBorrowerHomeowner', 'CreditScoreRangeLower', 'CreditScoreRangeUpper', 'DebtToIncomeRatio', 'IncomeRange', 
        'LoanOriginalAmount', 'LoanOriginationQuarter', 'PercentFunded', 'InvestmentFromFriendsAmount', 'BorrowerState']

In [6]:
# new high-level overview of data shape and composition
df = df[cols]
display(df.shape)
display(df.sample(5))

(113937, 19)

,Term,LoanStatus,BorrowerAPR,BorrowerRate,EstimatedReturn,ProsperRating (Alpha),ListingCategory (numeric),EmploymentStatus,EmploymentStatusDuration,IsBorrowerHomeowner,CreditScoreRangeLower,CreditScoreRangeUpper,DebtToIncomeRatio,IncomeRange,LoanOriginalAmount,LoanOriginationQuarter,PercentFunded,InvestmentFromFriendsAmount,BorrowerState
12396,36,Current,0.22147,0.1840,0.08172,C,1,Employed,30.0,False,640.0,659.0,0.09,"$25,000-49,999",5000,Q1 2014,1.0,0.0,MO
17635,60,Completed,0.28324,0.2573,0.14870,D,1,Employed,27.0,False,740.0,759.0,0.38,"$25,000-49,999",8000,Q3 2012,1.0,0.0,WA
1183,60,Current,0.27257,0.2469,0.09900,D,15,Employed,138.0,False,700.0,719.0,0.29,"$25,000-49,999",4000,Q4 2013,1.0,0.0,MO
102760,60,Current,0.25789,0.2326,0.14490,C,1,Employed,76.0,True,700.0,719.0,0.50,"$50,000-74,999",13000,Q3 2012,1.0,0.0,IL
43146,36,Current,0.17969,0.1435,0.07400,B,1,Employed,1.0,True,680.0,699.0,0.30,"$75,000-99,999",25000,Q4 2013,1.0,0.0,NY


In [7]:
# view data info()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 19 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Term                         113937 non-null  int64  
 1   LoanStatus                   113937 non-null  object 
 2   BorrowerAPR                  113912 non-null  float64
 3   BorrowerRate                 113937 non-null  float64
 4   EstimatedReturn              84853 non-null   float64
 5   ProsperRating (Alpha)        84853 non-null   object 
 6   ListingCategory (numeric)    113937 non-null  int64  
 7   EmploymentStatus             111682 non-null  object 
 8   EmploymentStatusDuration     106312 non-null  float64
 9   IsBorrowerHomeowner          113937 non-null  bool   
 10  CreditScoreRangeLower        113346 non-null  float64
 11  CreditScoreRangeUpper        113346 non-null  float64
 12  DebtToIncomeRatio            105383 non-null  float64
 13 

A little wrangling is required of this dataset. Some observed issues include:
1. Some feature names are not uniformly named.
2. Some features contains null values.
3. Some features should be compressed into a single feature column.
4. Some features are better represented as ordinal categorical datatypes while some other features datatypes should be changed.

I will be using the `Define - Code - Test` approach in cleaning the dataset.

### Issue 1 
#### Define
Some feature names are not uniformly named.

Rename feature names containing special characters.

In [8]:
# code
df_clean = df.copy()
df_clean.rename(columns={'ProsperRating (Alpha)':'ProsperRating', 'ListingCategory (numeric)':'ListingCategory'}, 
                inplace=True)

In [9]:
# test
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 19 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Term                         113937 non-null  int64  
 1   LoanStatus                   113937 non-null  object 
 2   BorrowerAPR                  113912 non-null  float64
 3   BorrowerRate                 113937 non-null  float64
 4   EstimatedReturn              84853 non-null   float64
 5   ProsperRating                84853 non-null   object 
 6   ListingCategory              113937 non-null  int64  
 7   EmploymentStatus             111682 non-null  object 
 8   EmploymentStatusDuration     106312 non-null  float64
 9   IsBorrowerHomeowner          113937 non-null  bool   
 10  CreditScoreRangeLower        113346 non-null  float64
 11  CreditScoreRangeUpper        113346 non-null  float64
 12  DebtToIncomeRatio            105383 non-null  float64
 13 

### Issue 2
#### Define
Some features in the dataset contains null values.

Drop all records that contains null values.

(I understand that in a real work environment it may be more expedient to understand why we have such a large number of null values and then find an innovative way of fixing such. But for this project, I will assume that it is impossible to get the missing data and that dropping null records instead of inputation or any other method for treating null values would not adversely affect my work) 

In [10]:
# code
df_clean.dropna(inplace=True)

# ListingCategory of 0 means that the listing has no specified category (about 19 have listing == 0; would drop them) 
df_clean[df_clean.ListingCategory == 0].count()

Term                           19
LoanStatus                     19
BorrowerAPR                    19
BorrowerRate                   19
EstimatedReturn                19
ProsperRating                  19
ListingCategory                19
EmploymentStatus               19
EmploymentStatusDuration       19
IsBorrowerHomeowner            19
CreditScoreRangeLower          19
CreditScoreRangeUpper          19
DebtToIncomeRatio              19
IncomeRange                    19
LoanOriginalAmount             19
LoanOriginationQuarter         19
PercentFunded                  19
InvestmentFromFriendsAmount    19
BorrowerState                  19
dtype: int64

In [11]:
# drop all records with a ListingCategory of 0 
df_clean = df_clean[df_clean.ListingCategory != 0]

In [12]:
# test
display(df_clean[df_clean.ListingCategory == 0])
df_clean.info()

,Term,LoanStatus,BorrowerAPR,BorrowerRate,EstimatedReturn,ProsperRating,ListingCategory,EmploymentStatus,EmploymentStatusDuration,IsBorrowerHomeowner,CreditScoreRangeLower,CreditScoreRangeUpper,DebtToIncomeRatio,IncomeRange,LoanOriginalAmount,LoanOriginationQuarter,PercentFunded,InvestmentFromFriendsAmount,BorrowerState


<class 'pandas.core.frame.DataFrame'>
Int64Index: 77524 entries, 1 to 113936
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Term                         77524 non-null  int64  
 1   LoanStatus                   77524 non-null  object 
 2   BorrowerAPR                  77524 non-null  float64
 3   BorrowerRate                 77524 non-null  float64
 4   EstimatedReturn              77524 non-null  float64
 5   ProsperRating                77524 non-null  object 
 6   ListingCategory              77524 non-null  int64  
 7   EmploymentStatus             77524 non-null  object 
 8   EmploymentStatusDuration     77524 non-null  float64
 9   IsBorrowerHomeowner          77524 non-null  bool   
 10  CreditScoreRangeLower        77524 non-null  float64
 11  CreditScoreRangeUpper        77524 non-null  float64
 12  DebtToIncomeRatio            77524 non-null  float64
 13  IncomeRange    

### Issue 3
#### Define
Some features should be compressed into a single feature column.

Combine `CreditScoreRangeUpper` and `CreditScoreRangeLower` features into a single feature column `CreditScoreRange` that should be an ordered categorial datatype

In [13]:
# code
# view the range of values
print(df_clean.CreditScoreRangeLower.sort_values().unique())
print(df_clean.CreditScoreRangeUpper.sort_values().unique())

# convert the dtype to int so as to remove the floting point zeros
df_clean = df_clean.astype({'CreditScoreRangeLower':'int', 'CreditScoreRangeUpper':'int'})

# convert the dtype to str so as to better join the values
df_clean = df_clean.astype({'CreditScoreRangeLower':'str', 'CreditScoreRangeUpper':'str'})

# create a new feature by
df_clean['CreditScoreRange'] = df_clean.CreditScoreRangeLower + '-' + df_clean.CreditScoreRangeUpper

# drop the base columns
df_clean.drop(labels=['CreditScoreRangeLower', 'CreditScoreRangeUpper'], axis=1, inplace=True)

[600. 620. 640. 660. 680. 700. 720. 740. 760. 780. 800. 820. 840. 860.
 880.]
[619. 639. 659. 679. 699. 719. 739. 759. 779. 799. 819. 839. 859. 879.
 899.]


In [14]:
# test
display(df_clean.head(2))
df_clean.info()

,Term,LoanStatus,BorrowerAPR,BorrowerRate,EstimatedReturn,ProsperRating,ListingCategory,EmploymentStatus,EmploymentStatusDuration,IsBorrowerHomeowner,DebtToIncomeRatio,IncomeRange,LoanOriginalAmount,LoanOriginationQuarter,PercentFunded,InvestmentFromFriendsAmount,BorrowerState,CreditScoreRange
1,36,Current,0.12016,0.0920,0.0547,A,2,Employed,44.0,False,0.18,"$50,000-74,999",10000,Q1 2014,1.0,0.0,CO,680-699
3,36,Current,0.12528,0.0974,0.0600,A,16,Employed,113.0,True,0.15,"$25,000-49,999",10000,Q4 2012,1.0,0.0,GA,800-819


<class 'pandas.core.frame.DataFrame'>
Int64Index: 77524 entries, 1 to 113936
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Term                         77524 non-null  int64  
 1   LoanStatus                   77524 non-null  object 
 2   BorrowerAPR                  77524 non-null  float64
 3   BorrowerRate                 77524 non-null  float64
 4   EstimatedReturn              77524 non-null  float64
 5   ProsperRating                77524 non-null  object 
 6   ListingCategory              77524 non-null  int64  
 7   EmploymentStatus             77524 non-null  object 
 8   EmploymentStatusDuration     77524 non-null  float64
 9   IsBorrowerHomeowner          77524 non-null  bool   
 10  DebtToIncomeRatio            77524 non-null  float64
 11  IncomeRange                  77524 non-null  object 
 12  LoanOriginalAmount           77524 non-null  int64  
 13  LoanOrigination

### Issue 4
#### Define
Some features are better represented as ordinal categorical datatypes while some other features datatypes should be changed.

Change the following features `Term`, `ProsperRating`, `ListingCategory`, `LoanStatus`, `EmploymentStatus`, `CreditScoreRange`, `IncomeRange`, `LoanOriginationQuarter` into categorical datatype. Also update the datatype of `EmploymentStatusDuration` from `float` to `int`.

In [15]:
# code
# convert to categorical
def convert_to_cat(a, b=True):
    '''
    Converts columns to categorical columns from a given dict
    
    Args:
    a: dict containing cols to be converted to categorical datatype
    b: status of the conversion if ordered or not
    
    Returns:
    None
    '''
    for var in a:
        ordered_var = CategoricalDtype(ordered=b, categories=a[var])
        df_clean[var] = df_clean[var].astype(ordered_var)
    

In [16]:
# convert Term, ProsperScore, ProsperRating, CreditScoreRange, IncomeRange, and LoanOriginationQuarter
# into ordered categorical types

# change ListingCategory numerical values to their corresponding textual representation
df_clean.ListingCategory = df_clean.ListingCategory.replace({1:'Debt Consolidation', 2:'Home Improvement', 3:'Business', 
                                                             4:'Personal Loan', 5:'Student Use', 6:'Auto', 7:'Other', 
                                                             8:'Baby & Adoption', 9:'Boat', 10:'Cosmetic Procedure', 
                                                             11:'Engagement Ring', 12:'Green Loans', 
                                                             13:'Household Expenses', 14:'Large Purchases', 
                                                             15:'Medical Dental', 16:'Motorcycle', 17:'RV', 18:'Taxes', 
                                                             19:'Vacation', 20:'Wedding Loans'})

# convert Term to a str data since I am not performing calculations on the terms and would be converting it to categorical
df_clean = df_clean.astype({'Term':'str'})

# choose the columns to be ordered and unordered categorical variables
ordinal_var_dict = {'Term': ['12', '36', '60'],
                    'ProsperRating': ['HR', 'E', 'D', 'C', 'B', 'A', 'AA'],
                    'CreditScoreRange': ['600-619', '620-639', '640-659', '660-679', '680-699', '700-719', '720-739',
                                         '740-759', '760-779', '780-799', '800-819', '820-839', '840-859', '860-879',
                                         '880-899'],
                    'IncomeRange': ['Not employed', '$1-24,999', '$25,000-49,999', '$50,000-74,999', '$75,000-99,999',
                                    '$100,000+'],
                    'LoanOriginationQuarter': ['Q3 2009', 'Q4 2009', 'Q1 2010', 'Q2 2010', 'Q3 2010', 'Q4 2010', 'Q1 2011',
                                               'Q2 2011', 'Q3 2011', 'Q4 2011', 'Q1 2012', 'Q2 2012', 'Q3 2012', 'Q4 2012',
                                               'Q1 2013', 'Q2 2013', 'Q3 2013', 'Q4 2013', 'Q1 2014']}
convert_to_cat(ordinal_var_dict)

# convert ListingCategory, EmploymentStatus, and LoanStatus into nominal categorical types
nominal_var_dict = {'ListingCategory': ['Debt Consolidation', 'Home Improvement', 'Business', 'Personal Loan',
                                             'Student Use', 'Auto', 'Other', 'Baby & Adoption', 'Boat', 
                                             'Cosmetic Procedure', 'Engagement Ring', 'Green Loans', 
                                             'Household Expenses', 'Large Purchases', 'Medical Dental', 
                                             'Motorcycle', 'RV', 'Taxes', 'Vacation', 'Wedding Loans'],
                    'EmploymentStatus': ['Employed', 'Full-time', 'Not employed', 'Other', 'Part-time', 
                                         'Retired', 'Self-employed'],
                    'LoanStatus': ['Chargedoff', 'Completed', 'Current', 'Defaulted', 'FinalPaymentInProgress',
                                   'Past Due (1-15 days)', 'Past Due (16-30 days)', 'Past Due (31-60 days)', 
                                   'Past Due (61-90 days)', 'Past Due (91-120 days)', 'Past Due (>120 days)']}
convert_to_cat(nominal_var_dict, False)

# convert the EmploymentStatusDuration dtype to int so as to remove the floting point zeros
df_clean = df_clean.astype({'EmploymentStatusDuration':'int64'})

# reset the index
df_clean.reset_index(drop=True, inplace=True)

In [17]:
# test
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77524 entries, 0 to 77523
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   Term                         77524 non-null  category
 1   LoanStatus                   77524 non-null  category
 2   BorrowerAPR                  77524 non-null  float64 
 3   BorrowerRate                 77524 non-null  float64 
 4   EstimatedReturn              77524 non-null  float64 
 5   ProsperRating                77524 non-null  category
 6   ListingCategory              77524 non-null  category
 7   EmploymentStatus             77524 non-null  category
 8   EmploymentStatusDuration     77524 non-null  int64   
 9   IsBorrowerHomeowner          77524 non-null  bool    
 10  DebtToIncomeRatio            77524 non-null  float64 
 11  IncomeRange                  77524 non-null  category
 12  LoanOriginalAmount           77524 non-null  int64   
 13  L

### What is the structure of your dataset?
This cleaned dataset contains 77,524 non null records and 18 features. The features contains 8 `categorical` datatypes, 6 `float` datatypes, and 2 `int` datatypes, 1 `str` datatype, and 1 `bool` datatype.

### What is/are the main feature(s) of interest in your dataset?
Since the dataset contains a large number of features, I am interested in evaluating the relationship between about 15-20 of the features; with a view of understanding the features that most likely affects the loan outcome, the borrower's interest rate, and the loan amount.

### What features in the dataset do you think will help support your investigation into your feature(s) of interest?
I expect that the credit and rating scores, and economic status (i.e.; employment status, home ownership, debt-to-income etc) would be pivotal in determining a loan outcome, borrower's interest rate, and loan amount.